<a href="https://colab.research.google.com/github/seungkyoon/Projects/blob/master/Bong_Seungkyoon_ps3_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PSET 3
*CPSC 453: Unsupervised Learning*

*Seungkyoon Bong*

In [2]:
from google.colab import drive, files
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
# ps3_functions.py (code copied below)
# from code.ps3_functions import *

# ps3_functions.py
# CPSC 453 -- Problem Set 3
#
# This script contains pytorch shells for a Logistic regression model, a feed forward network, and an autoencoder.
#
from torch.nn.functional import softmax
from torch import optim, nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import torch

class LogisticRegression(nn.Module): # initialize a pytorch neural network module
    def __init__(self): # initialize the model
        super(LogisticRegression, self).__init__() # call for the parent class to initialize
        # you can define variables here that apply to the entire model (e.g. weights, biases, layers...)
        # this model only has two parameters: the weight, and the bias.
        # here's how you can initialize the weight:
        # W = nn.Parameter(torch.zeros(shape)) # this creates a model parameter out of a torch tensor of the specified shape
        # ... torch.zeros is much like numpy.zeros, except optimized for backpropogation. We make it a model parameter and so it will be updated by gradient descent.
        self.W = nn.Parameter(torch.zeros(784, 10))

        # create a bias variable here
        self.b = nn.Parameter(torch.zeros(10))

    def forward(self, x):
        """
        this is the function that will be executed when we call the logistic regression on data.
        INPUT:
            x, an MNIST image represented as a tensor of shape 784
        OUTPUT:
            softmax_probabilities, a tensor of shape 10
        """
        # put the logic here.
        softmax_probabilities = torch.matmul(x, self.W) + self.b[None, :]

        return softmax_probabilities


class FeedForwardNet(nn.Module):
    """ Simple feed forward network with one hidden layer."""
    # Here, you should place an exact copy of the code from the LogisticRegression class, with two modifications:
    # 1. Add another weight and bias vector to represent the hidden layer
    # 2. In the forward function, add some type of nonlinearity to the output of the first layer, then pass it onto the hidden layer.
    def __init__(self, hidden_size=128, pre_activation = nn.Identity(), post_activation = nn.Identity()): # initialize the model
        super(FeedForwardNet, self).__init__()

        bound = 1/np.sqrt(hidden_size)
        W = torch.rand(784, hidden_size)*2*bound - bound
        self.W = nn.Parameter(W)
        b = torch.rand(hidden_size)*2*bound - bound
        self.b = nn.Parameter(b)

        self.pre_activation = pre_activation

        bound = 1/np.sqrt(10)
        W2 = torch.rand(hidden_size, 10)*2*bound - bound
        self.W2 = nn.Parameter(W2)
        b2 = torch.rand(10)*2*bound - bound
        self.b2 = nn.Parameter(b2)

        self.post_activation = post_activation

    def forward(self, x):
        """
        this is the function that will be executed when we call the logistic regression on data.
        INPUT:
            x, an MNIST image represented as a tensor of shape 784
        OUTPUT:
            softmax_probabilities, a tensor of shape 10
        """
        # put the logic here.
        hidden = torch.matmul(x, self.W) + self.b[None, :]
        activated = self.pre_activation(hidden)

        second_out = torch.matmul(activated, self.W2) + self.b2[None, :]
        softmax_probabilities = self.post_activation(second_out)

        return softmax_probabilities


class Autoencoder(nn.Module):
    def __init__(self, activation = None, sizes = [784, 1000, 500, 250, 2]):
        super(Autoencoder, self).__init__()
        self.lin1 = nn.Linear(sizes[0], sizes[1])
        # define additional layers here
        self.lin2 = nn.Linear(sizes[1], sizes[2])
        self.lin3 = nn.Linear(sizes[2], sizes[3])
        self.lin4 = nn.Linear(sizes[3], sizes[4])

        self.delin1 = nn.Linear(sizes[4], sizes[3])
        self.delin2 = nn.Linear(sizes[3], sizes[2])
        self.delin3 = nn.Linear(sizes[2], sizes[1])
        self.delin4 = nn.Linear(sizes[1], sizes[0])

        self.activation = activation
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        x = self.tanh(self.lin1(x))
        x = self.tanh(self.lin2(x))
        x = self.tanh(self.lin3(x))
        x = self.lin4(x)
        if self.activation:
            x = self.activation(x)

        return x

    def decode(self, z):
        z = self.tanh(self.delin1(z))
        z = self.tanh(self.delin2(z))
        z = self.tanh(self.delin3(z))
        z = self.sigmoid(self.delin4(z))

        return z

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)


# initialize the model (adapt this to each model)
model = LogisticRegression()
# initialize the optimizer, and set the learning rate
SGD = torch.optim.SGD(model.parameters(), lr = 42000000000) # This is absurdly high.
# initialize the loss function. You don't want to use this one, so change it accordingly
loss_fn = torch.nn.CrossEntropyLoss()
batch_size = 128

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


def train(model,loss_fn, optimizer, train_loader, test_loader):
    """
    This is a standard training loop, which leaves some parts to be filled in.
    INPUT:
    :param model: an untrained pytorch model
    :param loss_fn: e.g. Cross Entropy loss of Mean Squared Error.
    :param optimizer: the model optimizer, initialized with a learning rate.
    :param training_set: The training data, in a dataloader for easy iteration.
    :param test_loader: The testing data, in a dataloader for easy iteration.
    """
    num_epochs = 100 # obviously, this is too many. I don't know what this author was thinking.
    for epoch in range(num_epochs):
        # loop through each data point in the training set
        for data, targets in train_loader:

            # run the model on the data
            targets = targets.to(device)
            model_input = torch.reshape(data, (-1, 784)).to(device)# TODO: Turn the 28 by 28 image tensors into a 784 dimensional tensor.
            out = model(model_input)

            # Calculate the loss
            # targets = targets[:,None] # add an extra dimension to keep CrossEntropy happy.
            loss = loss_fn(out,targets)

            # Find the gradients of our loss via backpropogation
            loss.backward()

            # Adjust accordingly with the optimizer
            optimizer.step()
            optimizer.zero_grad()

        # Give status reports every 10 epochs
        if (epoch + 1) % 10==0:
            print(f" EPOCH {epoch+1}. Progress: {(epoch+1)/num_epochs*100}%. ")
            print(f" Train accuracy: {evaluate(model,train_loader)}. Test accuracy: {evaluate(model,test_loader)}") #TODO: implement the evaluate function to provide performance statistics during training.

def evaluate(model, evaluation_set):
    """
    Evaluates the given model on the given dataset.
    Returns the percentage of correct classifications out of total classifications.
    """
    total_correct = 0
    total_samples = 0

    with torch.no_grad(): # this disables backpropogation, which makes the model run much more quickly.
        # TODO: Fill in the rest of the evaluation function.
        for data, targets in evaluation_set:

            targets = targets.to(device)
            model_input = torch.reshape(data, (-1, 784)).to(device)
            out = model(model_input)

            total_samples += targets.shape[0]

            predictions = torch.argmax(out, 1)
            correct = torch.sum(predictions == targets)
            total_correct += correct


    accuracy = 1.0 * total_correct / total_samples
    return accuracy

# ----- Functions for Part 5 -----
def mmd(X,Y, kernel_fn):
    """
    Implementation of Maximum Mean Discrepancy.
    :param X: An n x 1 numpy vector containing the samples from distribution 1.
    :param Y: An n x 1 numpy vector containing the samples from distribution 2.
    :param kernel_fn: supply the kernel function to use.
    :return: the maximum mean discrepancy:
    MMD(X,Y) = Expected value of k(X,X) + Expected value of k(Y,Y) - Expected value of k(X,Y)
    where k is a kernel function
    """

    return mmd


def kernel(A, B):
    """
    A gaussian kernel on two arrays.
    :param A: An n x d numpy matrix containing the samples from distribution 1
    :param B: An n x d numpy matrix containing the samples from distribution 2.
    :return K:  An n x n numpy matrix k, in which k_{i,j} = e^{-||A_i - B_j||^2/(2*sigma^2)}
    """

    return K



In [ ]:
# GAN.py (code copied below)
# from code.GAN import *

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as datasets
import imageio
import numpy as np
import matplotlib
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
matplotlib.style.use('ggplot')

class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.nz = nz # the dimension of the random noise used to seed the Generator
        self.main = nn.Sequential( # nn.sequential is a handy way of combining multiple layers.
            nn.Linear(self.nz, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_input = 784
        self.main = nn.Sequential(
            nn.Linear(self.n_input, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        x = x.view(-1, 784)
        return self.main(x)


def train_discriminator(optimizer, real_data, fake_data):
    """
    Train the discriminator on a minibatch of data.
    INPUTS
        :param optimizer: the optimizer used for training
        :param real_data: the batch of training data
        :param fake_data: the data generated by the generator from random noise
    The discriminator will incur two losses: one from trying to classify the real data, and another from classifying the fake data.
    TODO: Fill in this function.
    It should
    1. Run the discriminator on the real_data and the fake_data
    2. Compute and sum the respective loss terms (described in the assignment)
    3. Backpropogate the loss (e.g. loss.backward()), and perform optimization (e.g. optimizer.step()).
    """

    # we'll return the loss for book-keeping purposes. (E.g. if you want to make plots of the loss.)
    return loss

def train_generator(optimizer, fake_data):
    """
    Performs a single training step on the generator.
    :param optimizer: the optimizer
    :param fake_data: forgeries, created by the generator from random noise. (Done before calling this function.)
    :return:  the generator's loss
    TODO: Fill in this function
    It should
    1. Run the discriminator on the fake_data
    2. compute the resultant loss for the generator (as described in the assignment)
    3. Backpropagate the loss, and perform optimization
    """
    return loss

# import data
batch_size = 100
train_data = datasets.MNIST(
    root='../data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

num_epochs = 1000
nz = # dimension of random noise
generator = Generator(nz)
discriminator = Discriminator(nz)

#TODO: Build a training loop for the GAN
# For each epoch, you'll
# 1. Loop through the training data. For each batch, feed random noise into the generator to generate fake_data of the corresponding size.
# 2. Feed the fake data and real data into the train_discriminator and train_generator functions
# At the end of each epoch, use the below functions to save a grid of generated images.
for epoch in range(num_epochs):
    for data in train_loader:
        # perform training

    # reshape the image tensors into a grid
    generated_img = make_grid(generated_img)
    # save the generated torch tensor images
    save_image(generated_img, f"../outputs/gen_img{epoch}.png")

In [ ]:
# vae.py (code copied below) 
# from code.vae import *

import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

parser = argparse.ArgumentParser(description='VAE MNIST Example') # collect arguments passed to file
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu") # Use NVIDIA CUDA GPU if available

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


def VAE_loss_function(recon_x, x, mu, logvar):
    # TO DO: Implement reconstruction + KL divergence losses summed over all elements and batch

    # see lecture 12 slides for more information on the VAE loss function
    # for additional information on computing KL divergence
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    


    return recon_loss + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = VAE_loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


if __name__ == "__main__":
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

## 2 MNIST Classification

### 2.1 Downloading MNIST

In [4]:
import torch
from torchvision import datasets, transforms

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

mnist_train = datasets.MNIST(root = 'data', train=True, download=True, transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = 'data', train=False, download=True,transform = transforms.ToTensor())

batch_size = 128
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size, shuffle = True)

### 2.2 Logistic Regression

In [7]:
model = LogisticRegression()
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.5)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/log_reg.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.926966667175293. Test accuracy: 0.9231999516487122
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.9321666955947876. Test accuracy: 0.9259999990463257
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.9296166896820068. Test accuracy: 0.9214999675750732
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9334666728973389. Test accuracy: 0.924299955368042
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9327666759490967. Test accuracy: 0.924299955368042
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.9330833554267883. Test accuracy: 0.9226999878883362
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.93586665391922. Test accuracy: 0.9235999584197998
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9366833567619324. Test accuracy: 0.9235000014305115
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.9358833432197571. Test accuracy: 0.9257999658584595
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.9327666759490967. Test accuracy: 0.9238999485969543


**Question 2.2.1:** *What percentage classification accuracy did your simple network achieve?*

**Answer:** 


### 2.3 Feed-forward Neural Network

In [34]:
# Without non-linearity
model = FeedForwardNet()
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_nonlinear.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.9136833548545837. Test accuracy: 0.9170999526977539
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.9216166734695435. Test accuracy: 0.9210999608039856
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.9241333603858948. Test accuracy: 0.9233999848365784
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9265000224113464. Test accuracy: 0.9232999682426453
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9276666641235352. Test accuracy: 0.9235999584197998
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.9292833209037781. Test accuracy: 0.9250999689102173
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.9298833608627319. Test accuracy: 0.9248999953269958
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9305166602134705. Test accuracy: 0.9259999990463257
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.9314166903495789. Test accuracy: 0.9261999726295471
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.931850016117096. Test accuracy: 0.9247999787330627

In [6]:
# With sigmoid non-linearity before and after
# Likely cripples the model to have a nonlinearity in output (because cross entropy loss already takes softmax)
model = FeedForwardNet(pre_activation = nn.Sigmoid(), post_activation = nn.Sigmoid())
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_sigmoid.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.8048833608627319. Test accuracy: 0.8134999871253967
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.836650013923645. Test accuracy: 0.8478999733924866
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.8513666987419128. Test accuracy: 0.8619999885559082
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.8617333173751831. Test accuracy: 0.8700999617576599
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.8689166903495789. Test accuracy: 0.8788999915122986
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.8741333484649658. Test accuracy: 0.8829999566078186
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.8786666989326477. Test accuracy: 0.8873999714851379
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.8823666572570801. Test accuracy: 0.8898999691009521
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.8849499821662903. Test accuracy: 0.8930999636650085
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.8878499865531921. Test accuracy: 0.8948999643325806

In [7]:
# With sigmoid non-linearity before
model = FeedForwardNet(pre_activation = nn.Sigmoid())
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_sigmoid_before.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.8827500343322754. Test accuracy: 0.8914999961853027
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.8986333608627319. Test accuracy: 0.9050999879837036
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.9065499901771545. Test accuracy: 0.9114999771118164
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9120833277702332. Test accuracy: 0.9174000024795532
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9163500070571899. Test accuracy: 0.9199000000953674
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.9205166697502136. Test accuracy: 0.9238999485969543
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.9232500195503235. Test accuracy: 0.9269999861717224
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9263499975204468. Test accuracy: 0.9287999868392944
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.9296333193778992. Test accuracy: 0.930899977684021
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.9322500228881836. Test accuracy: 0.9333999752998352

In [9]:
# With ReLU non-linearity before
model = FeedForwardNet(pre_activation = nn.ReLU())
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_relu.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.9298999905586243. Test accuracy: 0.9315999746322632
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.9463500380516052. Test accuracy: 0.9459999799728394
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.9550333619117737. Test accuracy: 0.9530999660491943
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9619166851043701. Test accuracy: 0.9581999778747559
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9670166969299316. Test accuracy: 0.9614999890327454
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.9704333543777466. Test accuracy: 0.964199960231781
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.9734333157539368. Test accuracy: 0.9667999744415283
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9755499958992004. Test accuracy: 0.9685999751091003
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.9776333570480347. Test accuracy: 0.9695999622344971
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.9796333312988281. Test accuracy: 0.9714999794960022

In [10]:
# With Softplus non-linearity before
model = FeedForwardNet(pre_activation = nn.Softplus())
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_softplus.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.9103500247001648. Test accuracy: 0.9124999642372131
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.9207500219345093. Test accuracy: 0.9210000038146973
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.9291333556175232. Test accuracy: 0.9284999966621399
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9354333281517029. Test accuracy: 0.9341999888420105
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9413166642189026. Test accuracy: 0.9391999840736389
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.9469833374023438. Test accuracy: 0.9430999755859375
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.9509666562080383. Test accuracy: 0.9475999474525452
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9554499983787537. Test accuracy: 0.9512999653816223
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.958466649055481. Test accuracy: 0.953499972820282
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.9616000056266785. Test accuracy: 0.9567999839782715


In [11]:
# Best activation function (ReLU) non-linearity with smaller hidden size
model = FeedForwardNet(hidden_size = 30, pre_activation = nn.ReLU())
model = model.to(device)
SGD = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, loss_fn, SGD, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ff_narrow.pt')

 EPOCH 10. Progress: 10.0%. 
 Train accuracy: 0.9064666628837585. Test accuracy: 0.906499981880188
 EPOCH 20. Progress: 20.0%. 
 Train accuracy: 0.921999990940094. Test accuracy: 0.9225999712944031
 EPOCH 30. Progress: 30.0%. 
 Train accuracy: 0.931950032711029. Test accuracy: 0.9307000041007996
 EPOCH 40. Progress: 40.0%. 
 Train accuracy: 0.9391999840736389. Test accuracy: 0.9375999569892883
 EPOCH 50. Progress: 50.0%. 
 Train accuracy: 0.9449833631515503. Test accuracy: 0.9434999823570251
 EPOCH 60. Progress: 60.0%. 
 Train accuracy: 0.948983371257782. Test accuracy: 0.9476999640464783
 EPOCH 70. Progress: 70.0%. 
 Train accuracy: 0.9531500339508057. Test accuracy: 0.9517999887466431
 EPOCH 80. Progress: 80.0%. 
 Train accuracy: 0.9565500020980835. Test accuracy: 0.9551999568939209
 EPOCH 90. Progress: 90.0%. 
 Train accuracy: 0.9593833684921265. Test accuracy: 0.9564999938011169
 EPOCH 100. Progress: 100.0%. 
 Train accuracy: 0.9616333246231079. Test accuracy: 0.9592999815940857


**Question 2.3.1:** *What percentage classification accuracy does this more complex network achieve?*

**Answer:** 

**Question 2.3.2:** *Create a plot of the training and test error vs the number of iterations. How many iterations
are sufficient to reach good performance?*

**Answer:** 

**Question 2.3.3:** *Print the confusion matrix showing which digits were misclassified, and what they were misclassified as. What numbers are frequently confused with one another by your model?*

**Answer:** 

**Question 2.3.4:** *Experiment with the learning rate, optimizer and activation function of your network. Report the best accuracy and briefly describe the training scheme that reached this accuracy.*

**Answer:** 


## 3 Autoencoder

### 3.1 MNIST

In [9]:
def train(model, loss_fn, optimizer, train_loader, test_loader):
    """
    This is a standard training loop, which leaves some parts to be filled in.
    INPUT:
    :param model: an untrained pytorch model
    :param loss_fn: e.g. Cross Entropy loss of Mean Squared Error.
    :param optimizer: the model optimizer, initialized with a learning rate.
    :param training_set: The training data, in a dataloader for easy iteration.
    :param test_loader: The testing data, in a dataloader for easy iteration.
    """
    num_epochs = 100 # obviously, this is too many. I don't know what this author was thinking.
    for epoch in range(num_epochs):
        # loop through each data point in the training set
        for data, targets in train_loader:

            # run the model on the data
            model_input = torch.reshape(data, (-1, 784)).to(device)# TODO: Turn the 28 by 28 image tensors into a 784 dimensional tensor.
            out = model(model_input)

            # Calculate the loss
            loss = loss_fn(out, model_input)

            # Find the gradients of our loss via backpropogation
            loss.backward()

            # Adjust accordingly with the optimizer
            optimizer.step()
            optimizer.zero_grad()

        # Give status reports every 5 epochs
        if (epoch + 1) % 5==0:
            print(f" EPOCH {epoch+1}. Progress: {(epoch+1)/num_epochs*100}%. ")
            print(f" Train loss: {evaluate(model,train_loader)}. Test loss: {evaluate(model,test_loader)}")

def evaluate(model, evaluation_set):
    """
    Evaluates the given model on the given dataset.
    Returns the MSE loss of the output and original input
    """
    total_loss = 0
    total_samples = 0
    loss_fn = torch.nn.MSELoss()

    with torch.no_grad(): # this disables backpropogation, which makes the model run much more quickly.
        # TODO: Fill in the rest of the evaluation function.
        for data, targets in evaluation_set:

            model_input = torch.reshape(data, (-1, 784)).to(device)
            out = model(model_input)

            total_samples += targets.shape[0]
            total_loss += loss_fn(out, model_input)*targets.shape[0]

    mean_loss = 1.0 * total_loss / total_samples
    return mean_loss

In [13]:
model = Autoencoder(activation = nn.Tanh())
model = model.to(device)
Adam = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = torch.nn.MSELoss()

train(model, loss_fn, Adam, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ae_tanh.pt')

 EPOCH 5. Progress: 5.0%. 
 Train loss: 0.03971928358078003. Test loss: 0.039753977209329605
 EPOCH 10. Progress: 10.0%. 
 Train loss: 0.03825227543711662. Test loss: 0.03825204819440842
 EPOCH 15. Progress: 15.0%. 
 Train loss: 0.03697613999247551. Test loss: 0.03711700811982155
 EPOCH 20. Progress: 20.0%. 
 Train loss: 0.03632295876741409. Test loss: 0.03646008297801018
 EPOCH 25. Progress: 25.0%. 
 Train loss: 0.035278987139463425. Test loss: 0.03563898056745529
 EPOCH 30. Progress: 30.0%. 
 Train loss: 0.03516751900315285. Test loss: 0.03558137267827988
 EPOCH 35. Progress: 35.0%. 
 Train loss: 0.03494458273053169. Test loss: 0.03547857701778412
 EPOCH 40. Progress: 40.0%. 
 Train loss: 0.03503577411174774. Test loss: 0.03564044088125229
 EPOCH 45. Progress: 45.0%. 
 Train loss: 0.03483179211616516. Test loss: 0.03520921617746353
 EPOCH 50. Progress: 50.0%. 
 Train loss: 0.03463738411664963. Test loss: 0.03501656651496887
 EPOCH 55. Progress: 55.00000000000001%. 
 Train loss: 0.034

In [14]:
import torchvision.utils
model = Autoencoder(activation = nn.Tanh())
model.load_state_dict(torch.load('/content/drive/MyDrive/ColabNotebooks/ae_tanh.pt'))
model.eval()

for data, targets in train_loader:
  model_input = torch.reshape(data, (-1, 784))
  out = model(model_input)
  out = out.reshape(-1, 28, 28)
  torchvision.utils.save_image(out[0, :, :], '/content/drive/MyDrive/ColabNotebooks/test_tanh.jpg')
  break

In [11]:
model = Autoencoder()
model = model.to(device)
Adam = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = torch.nn.MSELoss()

train(model, loss_fn, Adam, train_loader, test_loader)
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/ae_identity.pt')

 EPOCH 5. Progress: 5.0%. 
 Train loss: 0.03961634635925293. Test loss: 0.0399719662964344
 EPOCH 10. Progress: 10.0%. 
 Train loss: 0.036376673728227615. Test loss: 0.037006065249443054
 EPOCH 15. Progress: 15.0%. 
 Train loss: 0.035099174827337265. Test loss: 0.035776104778051376
 EPOCH 20. Progress: 20.0%. 
 Train loss: 0.03458607196807861. Test loss: 0.03539487347006798
 EPOCH 25. Progress: 25.0%. 
 Train loss: 0.034379735589027405. Test loss: 0.035191748291254044
 EPOCH 30. Progress: 30.0%. 
 Train loss: 0.033789992332458496. Test loss: 0.034768685698509216
 EPOCH 35. Progress: 35.0%. 
 Train loss: 0.033441927284002304. Test loss: 0.03448091819882393
 EPOCH 40. Progress: 40.0%. 
 Train loss: 0.032894037663936615. Test loss: 0.03393232822418213
 EPOCH 45. Progress: 45.0%. 
 Train loss: 0.03251643851399422. Test loss: 0.03377606347203255
 EPOCH 50. Progress: 50.0%. 
 Train loss: 0.032349176704883575. Test loss: 0.03374491631984711
 EPOCH 55. Progress: 55.00000000000001%. 
 Train los

In [12]:
import torchvision.utils
model = Autoencoder(activation = nn.Identity())
model.load_state_dict(torch.load('/content/drive/MyDrive/ColabNotebooks/ae_identity.pt'))
model.eval()

for data, targets in train_loader:
  model_input = torch.reshape(data, (-1, 784))
  out = model(model_input)
  out = out.reshape(-1, 28, 28)
  torchvision.utils.save_image(out[0, :, :], '/content/drive/MyDrive/ColabNotebooks/test_identity.jpg')
  break

**Question 3.1.1:** *Do the colors easily separate, or are they all clumped together? Which numbers are frequently embedded close together, and what does this mean?*

**Answer:** 

**Question 3.1.2:** *How realistic were the images you generated by interpolating between points in the latent space? Can you think of a better way to generate images with an autoencoder?*

**Answer:** 


### 3.2 Biological Data: Retinal Bipolar Dataset

**Question 3.2.1:** *How many clusters are visible in the embedding? Do they correspond to the cluster labels?*

**Answer:** 


## 4 Generative Models

### 4.1 The Variational Autoencoder

**Question 4.1.1:** *How does the VAE’s latent space compare to the latent space of your previous autoencoder? Do the generated images have more clarity? Is this most noticeable between or within classes?*

**Answer:** 

**Question 4.1.2:** *In what situations would a VAE be more useful than a vanilla autoencoder, and when would you prefer a vanilla autoencoder to a VAE?*

**Answer:** 

**Question 4.1.3:** *The distance between embeddings in your first autoencoder provided some measure of the similarity between digits. To what extent is this preserved, or improved, by the VAE?*

**Answer:** 


### 4.2 GANs

**Question 4.2.1:** *Which generates more realistic images: your GAN, or your VAE? Why do you think this
is?*

**Answer:** 

**Question 4.2.2:** *Does your GAN appear to generate all digits in equal number, or has it specialized in a smaller number of digits? If so, why might this be?*

**Answer:** 


## 5 Information Theory

### 5.1 Simple Distribution

**Question 5.1.1:** *Based on the above measures alone, which divergence seems most accurate?*

**Answer:** 


### 5.2 MNIST Sample Distributions

### 5.3 The GAN Distribution

**Question 5.3.1:** *Which divergence or distance showed the greatest discrepancy between the comparison between real MNIST data and the comparison with the GAN?*

**Answer:** 

**Question 5.3.2:** *Which of these information measures would you recommend for judging a GAN’s output? Why?*

**Answer:** 

**Question 5.3.3:** *How do the runtimes of these measures compare?*

**Answer:** 
